In [10]:
import os, sys, importlib
import rasterio

import geopandas as gpd
import pandas as pd
import numpy as np

from shapely.geometry import Point
from shapely.wkt import loads
from rasterio import features

sys.path.append("../")
import infrasap.vulnerability_mapping as vulmap

sys.path.append("/home/wb411133/Code/GOST")
import GOSTRocks.rasterMisc as rMisc


In [33]:
iso3 = "KEN"

global_bounds = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
pop_folder = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/GLOBAL_1km_Demographics"
pop_files = os.listdir(pop_folder)
temp_folder = "/home/wb411133/temp"
out_vulnerability = os.path.join(temp_folder, "%s_vulnerability_map.tif" % iso3)
out_pop = os.path.join(temp_folder, "%s_pop_map.tif" % iso3)


In [8]:
# extract national bounds
inG = gpd.read_file(global_bounds)
country_bounds = inG.loc[inG['ISO3'] == iso3]
country_bounds = country_bounds.to_crs({'init':'epsg:4326'})

In [15]:
# clip out the temporary vulnerability metrics
country_folder = os.path.join(temp_folder, iso3)
if not os.path.exists(country_folder):
    os.makedirs(country_folder)
for pFile in pop_files:
    curR = rasterio.open((os.path.join(pop_folder, pFile)))
    rMisc.clipRaster(curR, country_bounds, os.path.join(country_folder, pFile))

In [43]:
importlib.reload(vulmap)
# TODO: Run this with various parameters
vul_def = {'0-5'  :0.001,
           '5-10' :0.001,
           '10-15':0.001,
           '15-20':0.002,
           '20-25':0.005,
           '25-30':0.010,
           '30-35':0.016,
           '35-40':0.023,
           '40-45':0.029,
           '45-50':0.039,
           '50-55':0.058,
           '55-60':0.072,
           '60-65':0.102,
           '65-70':0.117,
           '70-75':0.146,
           '75-80':0.177,
           '80-100':0.180}
wp_folder = country_folder
wp_files  = [vulmap.wp_demographics(os.path.join(wp_folder, x)) for x in os.listdir(wp_folder)]
vul = vulmap.wp_vulnerability(wp_files, vul_def)

In [44]:
vul.calculate_vulnerability()
vul.calculate_total_pop()

In [46]:
vul.combine_results(out_vulnerability, out_pop)
